# Flight Streaming Data

This directory has code and schemas for interacting with the streaming data from the [OpenSky Network](https://opensky-network.org).

The python/flight/stream folder contains code that will pull the latest data from OpenSky and store the data in GCS and/or publish the data as messages in a Pub/Sub queue. The documentation for the OpenSky API is found at [https://openskynetwork.github.io/opensky-api/](https://openskynetwork.github.io/opensky-api/).

## How to Run Serveless

To run the code using a Cloud Function (aka. using serverless computing) you:

1. Configure your cloud function with permissions, time limits, and memory limits.
2. Supply the code and requirements.txt to the function. *This applies to a Cloud Function based on Python.*
3. Deploy the function.
4. Test the function with an example message.

You can then create a Big Query table from either the Cloud Storage bucket or the Pub/Sub queue using the provided schemas.

### Configure a Cloud Function

1. Environment: You can use either 1st Gen or 2nd Gen. 1st Gen has an easier interface for testing. 2nd Gen has the ability to have longer running cloud functions. (1st Gen will be limited to 10 mins.)
2. Function name: Choose a meaningful name. It can only have lowercase characters, numbers, and hyphens.
3. Trigger: Select HTTP/HTTPS. This means the function will be triggered whenever anyone hits a specific URL. (Other triggers are possible, such as triggering a function whenever a file is uploaded to cloud storage, a message is published in a Pub/Sub queue, and so on.)
4. Authentication: Allow unauthenticated invocations. Permissions often get hairy to set up properly, though you will want to enforce security constraints when working in a production system. For our purposes, we will not restrict access to the URL that is used to trigger the Cloud Function.
5. *(If using 1st Gen)* Click SAVE at the bottom of the Trigger box.
6. Runtime, build, and connection settings. You don't need to change the defaults. You may want to change the following: Memory allocated (if your function generates "Out of Memory" errors, but smaller memory allocation relates to smaller costs); Timeout (increase up to the maximum if your function fails to complete in time; smaller timeout means a function that hangs will be terminated more quickly); Autoscaling (lower the Maximum number of instances if you want to make sure the function doesn't spawn thousands of instances at one time when you have a lot of trigger events).

### Supply the Code

1. Select one of the Python 3 versions.
2. The entry point of the code is a method named flightStreaming. Change the default entry point from hello_world to flightStreaming.
3. Package up the code.

  A script is provided to package up the code for the flight streaming function. It creates a folder in /tmp/flight-streaming_zip and puts the subset of python files there, zips them up, and stores the zip file in Google Cloud Storage in a function subfolder in your bucket.

  You provide the script with the name of the cloud function, which is flight-streaming in this case.


You can browse Cloud Storage in the Cloud Function console to select the zip file.

You should see something like:

![Folder structure for Flight Streaming function](img/flightStreamingFolderStruct.png)

Once ready, select DEPLOY. It will take some time to create a virtual machine image and deploy your code into it. It will finish spinning and either have a green check mark or an error.

### Test Your Function

You can manually trigger your function using the Testing tab in the Cloud Function console (for 1st Gen) or copy-pasting a curl command in Cloud Shell (for 2nd Gen). In either case, you can supply a test such as:

See the openskyParser.py file for more examples.

## How to Run Locally *(for testing)*

To execute the Python code, you will need to follow the instructions in the README (displayed in the main page in the GitHub repository.) You only need to set up your Python environment if you want to run the code from Cloud Shell (to test it out without using Cloud Function.)

## Schemas

The schema folder has schemas used both for BigQuery tables as well as for connecting a Pub/Sub topic to a BigQuery table.
BigQuery schemas are used when you create a table and do not select to autodetect the schema. 

To create a table as an endpoint for messages published in a Pub/Sub topic will also need a schema that defines how to create columns out of the contents of the messages. The schema needed by DataFlow uses different types names than BigQuery.

**schema/openSky_bigQuery.json** -- a schema that can be used when loading data from Cloud Storage.

**schema/openSky_dataFlow.json** -- a schema that defines how to map the fields in a Pub/Sub message to columns.